# Setup

# Create Dataset

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv("/opt/gpudata/rrg-data-2/baselines/findings.csv")

In [ ]:
pd.read_csv("/opt/gpudata/rrg-data-2/baselines/impression.csv")

# Results

In [ ]:
findings_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inference_findings_data.csv")
impression_df = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inference_impression_data.csv")

print(findings_df.head())
print(impression_df.head())

find_sids = findings_df["study_id"].tolist()
imp_sids = impression_df["study_id"].tolist()

In [ ]:
# paths for generations

inference_results_path = "/opt/gpudata/rrg-data-2/inference-all/inf-results/"

## rgrg

In [ ]:
# rgrg_results = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/rgrg/generations_findings.csv")
rgrg_filename = "rgrg/generations_findings.csv"
rgrg_results = pd.read_csv(os.path.join(inference_results_path, rgrg_filename))
assert set(rgrg_results.study_id.tolist()) == set(find_sids)

## cxrmate

In [ ]:
cxrmate_results = pd.read_csv(os.path.join(inference_results_path, "cxr-mate/generations.csv"))
# cxrmate_results = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-mate/generations.csv")
cxrmate_findings = cxrmate_results[cxrmate_results["study_id"].isin(find_sids)][["study_id", "findings"]]

assert set(cxrmate_findings["study_id"]) == set(find_sids)
assert any(cxrmate_findings["findings"].notnull())

cxrmate_impression = cxrmate_results[cxrmate_results["study_id"].isin(imp_sids)][["study_id", "impression"]]

assert set(cxrmate_impression.study_id) == set(imp_sids)
assert any(cxrmate_impression["impression"].notnull())

cxrmate_both = pd.DataFrame.merge(cxrmate_findings, cxrmate_impression, on="study_id")
assert set(cxrmate_both.study_id) == set(set(find_sids).intersection(set(imp_sids)))

# findings
cxrmate_findings_final = cxrmate_findings.merge(findings_df, on="study_id")
cxrmate_findings_final.rename(columns={"findings_x" : "generated_text", "findings_y" : "actual_text"}, inplace=True)
# cxrmate_findings_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-mate/generations_findings.csv")
cxrmate_findings_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv(os.path.join(inference_results_path, "cxr-mate/generations_findings.csv"))


# impression
cxrmate_impression_final = cxrmate_impression.merge(impression_df, on="study_id")
cxrmate_impression_final.rename(columns={"impression_x" : "generated_text", "impression_y" : "actual_text"}, inplace=True)
# cxrmate_impression_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-mate/generations_impression.csv")
cxrmate_impression_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv(os.path.join(inference_results_path, "cxr-mate/generations_impression.csv"))


# both
def create_actual_report(row):
    return f"Findings:\n{row['findings']}\nImpression:\n{row['impression']}"

def create_generated_report(row):
    return f"Findings:\n{row['findings']}\nImpression:\n{row['impression']}"

full_reports_actual = pd.merge(findings_df, impression_df, on="study_id")
full_reports_actual["actual_text"] = full_reports_actual.apply(create_actual_report, axis=1)

# findings and impression
cxrmate_both["generated_text"] = cxrmate_both.apply(create_generated_report, axis=1)
cxrmate_both_final = cxrmate_both.merge(full_reports_actual[["study_id", "actual_text"]], on="study_id")
# cxrmate_both_final[["study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-mate/generations_full_reports.csv")
cxrmate_both_final[["study_id", "actual_text", "generated_text"]].to_csv(os.path.join(inference_results_path, "cxr-mate/generations_full_reports.csv"))

## chexagent

In [ ]:
# chexagent_results = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/chexagent/generations.csv")
chexagent_results = pd.read_csv(os.path.join(inference_results_path, "chexagent/generations.csv"))
chexagent_findings = chexagent_results[chexagent_results["study_id"].isin(find_sids)][["study_id", "findings"]]
chexagent_impression = chexagent_results[chexagent_results["study_id"].isin(imp_sids)][["study_id", "impression"]]
chexagent_both = pd.DataFrame.merge(chexagent_findings, chexagent_impression, on="study_id")

# findings

chexagent_findings_final = chexagent_findings.merge(findings_df, on="study_id")
chexagent_findings_final.rename(columns={"findings_x" : "generated_text", "findings_y" : "actual_text"}, inplace=True)
chexagent_findings_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/chexagent/generations_findings.csv")
chexagent_findings_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv(os.path.join(inference_results_path, "chexagent/generations_findings.csv"))

# impression

chexagent_impression_final = chexagent_impression.merge(impression_df, on="study_id")
chexagent_impression_final.rename(columns={"impression_x" : "generated_text", "impression_y" : "actual_text"}, inplace=True)
# chexagent_impression_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/chexagent/generations_impression.csv")
chexagent_impression_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv(os.path.join(inference_results_path, "chexagent/generations_impression.csv"))

# both

def create_actual_report(row):
    return f"Findings:\n{row['findings']}\nImpression:\n{row['impression']}"

def create_generated_report(row):
    return f"Findings:\n{row['findings']}\nImpression:\n{row['impression']}"

full_reports_actual = pd.merge(findings_df, impression_df, on="study_id")
full_reports_actual["actual_text"] = full_reports_actual.apply(create_actual_report, axis=1)

# findings and impression
chexagent_both["generated_text"] = chexagent_both.apply(create_generated_report, axis=1)
chexagent_both_final = chexagent_both.merge(full_reports_actual[["study_id", "actual_text"]], on="study_id")
# chexagent_both_final[["study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/chexagent/generations_full_reports.csv")
chexagent_both_final[["study_id", "actual_text", "generated_text"]].to_csv(os.path.join(inference_results_path, "chexagent/generations_full_reports.csv"))


## cxr-repair

In [ ]:
# cxr_repair_generations = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-repair/generations_full.csv")
cxr_repair_generations = pd.read_csv(os.path.join(inference_path_results, "cxr-repair/generations_full.csv"))

# filter and add actual text from impression
cxr_repair_gen_final = cxr_repair_generations.merge(impression_df, on="study_id")
cxr_repair_gen_final = cxr_repair_gen_final.drop_duplicates("study_id").rename(columns={"impression" : "actual_text"})
# cxr_repair_gen_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-repair/generations_impression.csv")
cxr_repair_gen_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv(os.path.join(inference_results_path, "cxr-repair/generations_impression.csv"))

## cxr-redone

In [ ]:
# cxr_redone_results = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-redone/generations.csv")
cxr_redone_results = pd.read_csv(os.path.join(inference_results_path, "cxr-redone/generations.csv"))

# xrem_sids = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/x-rem/xrem_sids.csv")
xrem_sids = pd.read_csv(os.path.join(inference_results_path, "x-rem/xrem_sids.csv"))
cxr_redone_results = cxr_redone_results.merge(xrem_sids, left_index=True, right_index=True, how="inner")
cxr_redone_final = cxr_redone_results.merge(impression_df, on="study_id")
cxr_redone_final = cxr_redone_final.drop_duplicates("study_id").rename(
    columns={
        "impression" : "actual_text",
        "Report Impression": "generated_text"
    })
# cxr_redone_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/cxr-redone/generations_impression.csv")
cxr_redone_final[["dicom_id", "study_id", "actual_text", "generated_text"]].to_csv(os.path.join(inference_results_path, "cxr-redone/generations_impression.csv"))



## xrem

In [ ]:
# xrem_results = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/x-rem/final_results_filtered.csv")
# xrem_sids = pd.read_csv("/opt/gpudata/rrg-data-2/inference-all/inf-temp/x-rem/xrem_sids.csv")
xrem_results = pd.read_csv(os.path.join(inference_results_path, "x-rem/final_results_filtered.csv"))
xrem_sids = pd.read_csv(os.path.join(inference_results_path, "x-rem/xrem_sids.csv"))
xrem_results_all = xrem_results.merge(xrem_sids, left_index=True, right_index=True, how="inner")
xrem_results_all = xrem_results_all.merge(impression_df, on="study_id")
xrem_results_all = xrem_results_all.drop_duplicates("study_id")
# xrem_results_all[["dicom_id", "study_id", "impression", "Report Impression"]].rename(
#     columns={
#         "impression" : "actual_text",
#         "Report Impression" : "generated_text"
#     }).to_csv("/opt/gpudata/rrg-data-2/inference-all/inf-results/x-rem/generations_impression.csv")
xrem_results_all[["dicom_id", "study_id", "impression", "Report Impression"]].rename(
    columns={
        "impression" : "actual_text",
        "Report Impression" : "generated_text"
    }).to_csv(os.path.join(inference_results_path, "x-rem/generations_impression.csv"))

# Create inference results

In [ ]:
def extract_exp_info(experiment):
    tabs = experiment.split("_")
    return {
         "section" : "_".join(tabs[1:-1])
    }

# extract_exp_info("generations_full_reports_METRICS.csv")
# extract_exp_info("generations_findings_METRICS.csv")

# our results
def extract_results(df):
    results = {}
    results["run"] = ""
    results["BLEU4"] = df["bleu4"].mean() * 100.0
    results["RougeL"]= df["rougeL"].mean() * 100.0
    results["BertScore"] = df["bertscore"].mean() * 100.0
    results["f1-RadGraph"] = df["f1radgraph"].mean() * 100.0
    results["f1-CheXbert"] = df["f1chexbert"].mean() * 100.0
    return results

In [ ]:
results_dir = "/opt/gpudata/rrg-data-2/inference-all/inf-results/"
all_results = {
    "findings" : [],
    "impression" : [],
    "both" : [],
}

for foldername in os.listdir(results_dir):
    print(foldername)
    folderpath = os.path.join(results_dir, foldername)
    if os.path.isdir(folderpath):
        for filename in os.listdir(folderpath):
            if "METRICS" in filename:
                res = extract_exp_info(filename)
                filepath = os.path.join(results_dir, os.path.join(foldername, filename))
                df = pd.read_csv(filepath)

                # aggregate results
                results = {}
                # run_name = f"{foldername}_{res['section']}"
                run_name = f"{foldername}"
                results["run"] = run_name
                results["BLEU4"] = df["bleu4"].mean() * 100.0
                results["RougeL"]= df["rougeL"].mean() * 100.0
                results["BertScore"] = df["bertscore"].mean() * 100.0
                results["f1-RadGraph"] = df["f1radgraph"].mean() * 100.0
                results["f1-CheXbert"] = df["f1chexbert"].mean() * 100.0

                if "findings" in filename:
                    all_results["findings"].append(results)
                elif "impression" in filename:
                    all_results["impression"].append(results)
                else:
                    all_results["both"].append(results)

### findings

In [ ]:
# findings
findings_best = pd.read_csv("/opt/gpudata/rrg-data-2/exp-findings/exp-model/Mistral-7B-Instruct-v0.3_exact_pred-label_simple_top-5_findings_METRICS.csv")
findings_result = extract_results(findings_best)
findings_result["run"] = "lab-rag"
pd.DataFrame(findings_result, index=[0])

In [ ]:
pd.DataFrame.from_dict(all_results["findings"])

In [ ]:
# final table
findings_df = pd.DataFrame.from_dict(all_results["findings"])
findings_df.loc[-1] = findings_result
findings_df.reset_index(drop=True)

### impression

In [ ]:
# impression
impression_best = pd.read_csv("/opt/gpudata/rrg-data-2/exp-impression/exp-model/Mistral-7B-Instruct-v0.3_exact_pred-label_simple_top-5_impression_METRICS.csv")
impression_result = extract_results(impression_best)
impression_result["run"] = "lab-rag"
pd.DataFrame(impression_result, index=[0])

In [ ]:
pd.DataFrame.from_dict(all_results["impression"])

In [ ]:
# final table
impression_df = pd.DataFrame.from_dict(all_results["impression"])
impression_df.loc[-1] = impression_result
impression_df.reset_index(drop=True)

### both

In [ ]:
#both 
both_best = pd.read_csv("/opt/gpudata/rrg-data-2/exp-section/Mistral-7B-Instruct-v0.3_exact_pred-label_simple_top-5_both_METRICS.csv")
both_result = extract_results(both_best)
both_result["run"] = "lab-rag"
pd.DataFrame(both_result, index=[0])

In [ ]:
pd.DataFrame.from_dict(all_results["both"])

In [ ]:
# final table

both_df = pd.DataFrame.from_dict(all_results["both"])
both_df.loc[-1] = both_result
both_df.reset_index(drop=True)